In [1]:
import os

In [2]:
%pwd

'c:\\Users\\karth\\Documents\\EtE_prj\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../") 

In [4]:
%pwd

'c:\\Users\\karth\\Documents\\EtE_prj\\Chicken-Disease-Classification'

In [5]:
#Data Ingection part. Data have to be injected or need to bring from yaml file. 

from dataclasses import dataclass 
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

    # DataIngestionConfig is an entity. if one function defined with return type as this entity( DataIngestionConfig),
    #  that function should return 4 variable values (with data type defined) as mentioned in entity
    # 

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

### Defining ConfigurationManager for bringing configuration info for Data Ingestion

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):   # Reading variable values which contains path to the yaml files

        self.config = read_yaml(config_filepath)# reading input file path
        self.params = read_yaml(params_filepath)  # Reading files from the paths

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir]) # creating a directory mentioned in root_dir 

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        ) #bringing data ingestion config information in format defined in DataIngestionConfig entity

        return data_ingestion_config  # returning a entity which is type DataIngestionConfig. Contains attributes for the data ingestion
      

In [10]:

import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

### Defining data ingestion class(downloading and unzipping the file)

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):# If zip file not exists, download
            filename, headers = request.urlretrieve( # It's downloads the zip file from url
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    # It unzips the file which downloaded from url 
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### Pipeline for Data Ingestion. It will Perform Data Ingestion in action

In [12]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-25 06:22:58,290: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-25 06:22:58,293: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-25 06:22:58,295: INFO: common: created directory at: artifacts]
[2024-02-25 06:22:58,296: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-25 06:23:02,766: INFO: 2055231800: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7438:27432B:C75B2:11BEB1:65DA8F68
Accept-Ranges: bytes
Date: Sun, 25 Feb 2024 00:52:59 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10223-MAA
X-Cache: M